In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

%load_ext autoreload 
%autoreload 2


In [2]:
# TORCH MNIST DISTRIBUTED EXAMPLE

"""run.py:"""
#!/usr/bin/env python
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp

from pyfig import Pyfig
import numpy as np

torch.manual_seed(1234)
torch.set_default_tensor_type(torch.DoubleTensor)   # ❗ Ensure works when default not set AND can go float32 or 64

arg = dict(
    charge = 0,
    spin  = 0,
    a = np.array([[0.0, 0.0, 0.0],]),
    a_z  = np.array([4.,]),
    n_b = 256, 
    n_sv = 32, 
    n_pv = 16, 
    n_corr = 40, 
    n_step = 10000, 
    log_metric_step = 5, 
    exp_name = 'demo',
    # sweep = {},
)

c = Pyfig(wb_mode='online', arg=arg, submit=False, run_sweep=False)

n_device = c.n_device
print(f'🤖 {n_device} GPUs available')



init sub classes
updating configuration
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: b90fa37 stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
ip not in ref
127.0.0.1 None
stdin not in ref
9003 None
control not in ref
9001 None
hb not in ref
9000 None
Session.signature_scheme not in ref
"hmac-sha256" None
Session.key not in ref
b"c667ba88-a2af-48da-bf6c-5128b19b532c" None
shell not in ref
9002 None
transport not in ref
"tcp" None
iopub not in ref
9004 None
f not in ref
/home/amawi/.local/share/jupyter/runtime/kernel-v2-11368P1V8MIP9lw74.json None
ip 127.0.0.1 not assigned
stdin 9003 not assigned
control 9001 not assigned
hb 9000 not assigned
Session.signature_scheme hmac-sha256 not assigned
Session.key b'c667ba88-a2af-48da-bf6c-5128b19b532c' not assigned
shell 9002 not assigned
transport tcp not assigned
iopub 9004 not assigned
f /home/amawi/.local/share/jupyter/runtime/kernel-v2-11368P1V8MIP9lw74.json not assigned
Run: ['git', 'log'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xmax1 (hwat). Use `wandb login --relogin` to force relogin


🤖 1 GPUs available


In [3]:
import torch
torch.manual_seed(1234)
torch.set_default_tensor_type(torch.DoubleTensor)   # ❗ Ensure works when default not set AND can go float32 or 64
n_device = torch.cuda.device_count()
print(n_device)
current_device = torch.cuda.current_device()
device = torch.cuda.device(0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device_name = torch.cuda.get_device_name(0)
print('cuda: ', device, 'n_dev: ', n_device, 'device', device, 'name: ', device_name)

1
cuda:  cuda n_dev:  1 device cuda name:  NVIDIA TITAN Xp


In [4]:

### model (aka Trainmodel) ### 
from hwat_func import Ansatz_fb
from torch import nn
from hwat_func import compute_ke_b, compute_pe_b
from hwat_func import init_r, get_center_points
import wandb
from hwat_func import keep_around_points, sample_b
from utils import compute_metrix

from functorch import vmap, make_functional, grad

_dummy = torch.randn((1,))
dtype = _dummy.dtype
device='cuda'
c._convert(device=device, dtype=dtype)
model = c.partial(Ansatz_fb).to(device).to(dtype)

center_points = get_center_points(c.data.n_e, c.data.a)
r = init_r(n_device, c.data.n_b, c.data.n_e, center_points, std=0.1)[0]
deltar = torch.tensor([0.02]).to(device).to(dtype)

# from torch.jit import Final    - https://pytorch.org/docs/stable/jit.html type safety and optimisations

# from torch.profiler import profile, record_function, ProfilerActivity

# with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
#     with record_function("model_inference"):
#         model(r[0])

# # print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
# print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

model(r[0])
mod = torch.jit.script(model)

r.requires_grad_(False)
mod(r[0])

Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: b90fa37 stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
False
torch.Size([1])
False
[1]


tensor(-16.7441, device='cuda:0', grad_fn=<SqueezeBackward0>)

In [ ]:

### fix sampler
### fix train step 
### metrix conversion

model_fn, params = make_functional(model)
# model_v = torch.compile(model_fn)
model_v = torch.jit.script(model_fn)
model_v = vmap(model_fn, in_dims=(None, 0))

model_v(params, r)


In [ ]:
opt = torch.optim.RAdam(model.parameters(), lr=0.0001)


wandb.define_metric("*", step_metric="tr/step")
for step in range(1, c.n_step+1):
    
    r, acc, deltar = sample_b(model_v, params, r, deltar, n_corr=c.data.n_corr)  # ❗needs testing 
    r = keep_around_points(r, center_points, l=2.) if step < 1000 else r
    
    model_ke = lambda _r: model_v(params, _r).sum()

    with torch.no_grad():
        ke = compute_ke_b(model_ke, r)
        pe = compute_pe_b(r, c.data.a, c.data.a_z)
        e = pe + ke
        e_mean_dist = torch.mean(torch.abs(torch.median(e) - e))
        e_clip = torch.clip(e, min=e-5*e_mean_dist, max=e+5*e_mean_dist)

    # opt.zero_grad()
    loss_fn = lambda _params: ((e_clip - e_clip.mean())*model_v(_params, r)).mean()
    
    grads = grad(loss_fn)(params)
    for p, g in zip(model.parameters(), grads):
        p.grad = torch.nn.utils.clip_grad_norm_(g.clone(), max_norm=1.)

    opt.step()
    
    params = [p.detach() for p in model.parameters()]
    grads = [p.grad.detach() for p in model.parameters()]
    
    v_tr = dict(
        params=params, grads=grads,
        e=e, pe=pe, ke=ke, r=r,
    )
    
    if not (step % c.log_metric_step):
        metrix = compute_metrix(v_tr)  # ❗ needs converting to torch, ie tree maps
        wandb.log({'tr/step':step, **metrix})